In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data.csv")

In [3]:
X = df.drop("Success", axis = 1)
X.insert(0, "Intercept", 1)
X = X.to_numpy()
y = df["Success"].to_numpy()

In [4]:
X

array([[ 1.        ,  1.48938133,  1.15341522],
       [ 1.        ,  1.81100853,  0.94496669],
       [ 1.        , -0.044533  ,  0.34278203],
       [ 1.        , -0.36616019,  1.13025428],
       [ 1.        ,  0.15339143, -0.79210443],
       [ 1.        , -1.60318788, -1.8343471 ],
       [ 1.        , -0.14349521, -0.67629969],
       [ 1.        , -0.44038186, -0.79210443],
       [ 1.        , -0.7372685 , -0.02779314],
       [ 1.        , -0.11875466,  0.55123057]])

## <font color = "brown"> Functions </font>

In [5]:
def g(x):
    
    # Link function
    return np.log(x / (1 - x))

def ig(x):
    
    # Inverse link function
    return 1 / (1 + np.exp(-x))

def dg(x):
    
    # Derivative of link function
    return 1 / (x * (1 - x))

In [6]:
def V(x):
    
    # Variance function of logistic regression
    return x * (1 - x)

In [7]:
def WLS(X, W, z):
    
    # Normal equations of weighted least squares
    XtWX = X.T @ W @ X
    XtWz = X.T @ W @ z
    b = np.linalg.inv(XtWX) @ XtWz
    
    return b

In [8]:
def Likelihood(p, y):
    
    # Likelihood of logistic regression
    L = y * np.log(p) + (1 - y) * np.log(1 - p)
    return L

## <font color = "brown"> IWLS Algorithm </font>

In [9]:
def IWLS(X, y):
    
    # Step 1
    # Initialise probability values of 0.5
    mu = np.ones(len(y)) / 2
    
    delta = 1
    i = 0
    LL = 0
    
    while delta > 1e-6:
        
        # Step 2
        # Given current μ, calculate z and W
        z = g(mu) + (y - mu) * dg(mu)
        W = np.diag(1 / (dg(mu)**2 * V(mu))) # Inverse of Sigma
        
        # Step 3
        # Given current z and W, calculate β
        b = WLS(X, W, z)
        
        # Step 4
        # Given current β, calculate μ
        mu = ig(X @ b)
        
        # Step 5
        # Calculate and compare log-likelihoods
        LLOld = LL
        LL = Likelihood(mu, y)
        delta = abs(LL - LLOld)
        
        i += 1
        
    print(f"IWLS completed on {i} iterations")
    return b

In [10]:
b = IWLS(X, y)

IWLS completed on 8 iterations


In [11]:
b

array([1.10999575, 9.12479957, 2.18745957])

In [12]:
phat = ig(X @ b)
yhat = np.where(phat >= 0.5, 1, 0)
yhat

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 1])

## <font color = "brown"> Comparison </font>

$$
\begin{aligned}
\\
\\
\hat\beta &= (X^TX)^{-1}(X^Ty)
\\
\\
\\
\epsilon &\sim N(0, \sigma^2I)
\\
\\
\\
\epsilon &\sim N(0, \Sigma)
\\
\\
\\
\hat\beta &= (X^T\Sigma^{-1}X)^{-1}(X^T\Sigma^{-1}y)
\\
\\
\\
g(\hat\mu) &= X\beta
\\
\\
\\
\hat\mu &= g^{-1}(X\beta)
\\
\\
\\
g(\mu) &= \log\left(\frac{\mu}{1-\mu}\right) \\
g'(p) &= \frac{1}{p(1-p)} \\
g^{-1}(\eta) &= \frac{1}{1 + \exp(-\eta)} \\
\\
\\
\\
f(y) &= \exp\left[\frac{\theta y - b(\theta)}{a(\phi)} + c(y, \phi)\right]
\\
\\
\\
\text{Var}(Y) &= v(\mu)a(\phi)
\\
\\
\\
v(\mu) &= b'' o (b')^{-1}(\mu)
\\
\\
\\
v(\mu) &= \mu(1 - \frac{\mu}{n})
\\
\\
\\
Z_i &= g(\mu_i) + \epsilon_i \\
\epsilon_i & = (Y_i - \mu_i)g'(\mu_i)
\\
\\
\\
\Sigma_{ii} &= (g'(\mu))^{2}\text{Var}(Y_i) \\
&= (g'(\mu))^{2}v(\mu_i)a(\phi)
\\
\\
\\
\hat\beta &= (X^T\Sigma^{-1}X)^{-1}(X^T\Sigma^{-1}z) 
\\
\\
\\
\hat\mu_i &= g^{-1}(x_i^T\hat\beta)
\\
\\
\\
\log\mathcal{L}(\beta) &= \sum y_i\log(p_i) + (1 - y_i)\log(1-p_i)
\\
\\
\\
\end{aligned}
$$


$$
\begin{aligned}
\\
\\
\\
f(y) &= \left(\begin{matrix}n \\ y\end{matrix}\right)p^y(1-p)^{n-y} \\
\log f(y) &= \log\left(\begin{matrix}n \\ y\end{matrix}\right) + y\log p + (n-y)\log(1-p) \\
f(y) &= \exp\left(\log\left(\begin{matrix}n \\ y\end{matrix}\right) + y\log p + n\log(1-p) - y\log(1-p) \right) \\
f(y) &= \exp\left(\log\left(\begin{matrix}n \\ y\end{matrix}\right) + y\log \frac{p}{1-p} + n\log(1-p) \right) \\
\\
\text{Let, }\space\space\space \theta &= \log\frac{p}{1-p},\space\space p = \frac{e^\theta}{1+e^\theta}, \space\space
1 - p = \frac{1}{1+e^\theta}
\\
\\
f(y) &= \exp\left(\theta y - n\log(1+e^\theta) + \log\left(\begin{matrix}n \\ y\end{matrix}\right)\right) 
\\
\\
\text{Let, }\space\space\space b(\theta) &= n\log(1+e^\theta),\space\space a(\phi) = 1, \space\space
c(y, \phi) = \log\left(\begin{matrix}n \\ y\end{matrix}\right) 
\\
\\
f(y) &= \exp\left(\frac{\theta y - b(\theta)}{a(\phi)} + c(y, \phi)\right)
\\
\\
\\
\end{aligned}
$$